# Lab 2: Linear models, neural networks and SVMs

Now the first lab is complete which recapped Python, Jupyter Notebooks and NumPy operations and introduced scikit-learn (go back over the lab if you're unsure about anything as this lab builds from it), it's time to implement some key supervised learning algorithms: linear regression, neural networks and support vector machines (SVMs).

## Importing the libraries

This lab requires `h5py` package to interact with a dataset that is stored in an H5 file and `imageio` & `PIL` packages for image processing. If you don't have these packages then install using pip3 or conda, as an example run:
```
$ conda install h5py
```

Firstly, we'll import the required packages by running the cell below.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.datasets import make_moons

import h5py
import imageio
from PIL import Image 
from utils import * # image processing functions from utils.py

## 1) Linear models for regression
Much of machine learning is about fitting functions to data and we begin with linear models, a class of models that are linear functions of the adjustable parameters. The simplest form of linear models are also linear functions of the input variables (simply known as linear regression). For example, for a 3-dimensional (D=3) input $\mathbf{x}=[x_1, x_2, x_3]^T$, linear regression is a linear combination of the input variables plus a constant $b$:

$$ f(\mathbf{x};\mathbf{w},b) = w_1 x_1 + w_2 x_2 + w_3 x_3 + b = \mathbf{w}^T \mathbf{x}  + b \qquad(1) $$

where $\mathbf{w}$ is a 3-dimensional vector of weights and the constant bias $b$ gives the value of the function at the origin.

We want to find the parameters, $\mathbf{w}$, of the linear function that best fits our training dataset of input-output pairs. We will first express our dataset of N examples as an NxD matrix called the *design matrix*, $X$, and the corresponding observed outputs into an Nx1 column vector, $\mathbf{y}$.

$$ \mathbf{y} = \left[ \begin{array}{c}y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(N)} \end{array} \right],
    \qquad
    X = \left[ \begin{array}{c}\mathbf{x}^{(1)\top} \\ \mathbf{x}^{(2)\top} \\ \vdots \\ \mathbf{x}^{(N)\top} \end{array} \right]
    = \left[ \begin{array}{cccc}
        x_1^{(1)} & x_2^{(1)} & \cdots & x_D^{(1)} \\
        x_1^{(2)} & x_2^{(2)} & \cdots & x_D^{(2)} \\
        \vdots & \vdots & \ddots & \vdots \\
        x_1^{(N)} & x_2^{(N)} & \cdots & x_D^{(N)} \\
\end{array} \right] $$

$$ \mathbf{f} = X\mathbf{w} + b \qquad(2) $$

Expressing the data in the form of a matrix and vector allows us to use the notation of linear algebra to derive the solution. This improves readability and maps more closely to how this is implemented efficiently in code with matrix-vector operations.

We can compute the total square error of the function values above, compared to the observed training set values: 

$$ \sum_{n=1}^N [y^{(n)} - f(\mathbf{x}^{(n)};\mathbf{w},b)]^2 = (\mathbf{y}-\mathbf{f})^T(\mathbf{y}-\mathbf{f}) \qquad(3) $$ 

The least-squares fitting problem is finding the parameters that minimise this error.

Note that there is a notational trick that allows for the bias term, $b$, to be omitted from equations 1 and 2 above. If we construct our design matrix to include an additional column/dimension (so that it is now NxD+1) containing a vector of 1's then the bias term can simply be interpreted  as another weight (i.e. $b = \mathbf{w}_{D+1}^{(n)}\mathbf{x}_{D+1}^{(n)}$ where $\mathbf{x}_{D+1}^{(n)} = 1$ for all $n$).

### 1.1) Least squares fitting

We will begin by generating a series of points from a given quadratic (non-linear) equation $y=(x-1)(x-5) = x^2-6x+5$ with normally distributed noise added i.e. $\mathcal{N}(\mu=0,\sigma=5)$.

In [0]:
np.random.seed(0)
N = 30
sigma = 5
x = np.sort(np.random.sample((N,1)))*10
y = (x-1)*(x-5) + np.random.normal(0,sigma,N).reshape(-1, 1)

fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x, y)
ax.set_xlabel('x')
ax.set_ylabel('y')

Fit a linear function to the generated data, print $w_1$ and $b$ (or $w_0$) and plot the learnt function.

As a helper:
- 1) Create the array $X_{bias}$ by concatenating a vector of ones to $X$ (use [np.concatenate](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html))
- 2) Calculate $w$ using [np.linalg.lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)
- 3) Generate predictions $y_{pred}$ for the fitted function
- 4) Plot the learnt linear function alongside the original data points.

In [0]:
# write your code here


Now use [scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares) to fit the linear model. Do you get the same $w_1$ and $b$ (coefficient and intercept)? Is the model overfitting or underfitting?

In [0]:
# write your code here


### 1.2) Non-linear regression: polynomial fitting
Remember that the linear regression model from exercise 1.1 was unable to fit the data well because the data was not linear. We  extend the class of linear models by considering linear combinations of a set of fixed non-linear functions, or basis functions, (see Bishop section 3.1 for more information on basis functions) applied to the input data. Examples of basis functions include the Gaussian basis function and sigmoid basis function but here we will use a polynomial basis function (see Bishop section 3.1 for more details of basis functions). The purpose for doing this is to transform the data into a higher dimensional space such that a linear function can be fit to it.

To fit a polynomial function, we use the following matrix $\Phi$ with the rows $\phi$ consisting of the polynomial basis function.

$$
\Phi = \left[ \begin{array}{ccccc}
        \phi_1(x^{(1)}) & \phi_2(x^{(1)}) & \phi_3(x^{(1)}) & \cdots & \phi_K(x^{(1)}) & \\
        \phi_1(x^{(2)}) & \phi_2(x^{(2)}) & \phi_3(x^{(2)}) & \cdots & \phi_K(x^{(2)}) & \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        \phi_1(x^{(N)}) & \phi_2(x^{(N)}) & \phi_3(x^{(N)}) & \cdots & \phi_K(x^{(N)}) & \\
        \end{array} \right] =
        \left[ \begin{array}{ccccc}
        1 & x^{(1)} & {(x^{(1)})}^2 & \cdots & {(x^{(1)})}^{K-1} \\
        1 & x^{(2)} & {(x^{(2)})}^2 & \cdots & {(x^{(2)})}^{K-1} \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        1 & x^{(N)} & {(x^{(N)})}^2 & \cdots & {(x^{(N)})}^{K-1} \\
        \end{array} \right]   
$$

$$ f(\mathbf{x}) = \mathbf{w}^T \phi (\mathbf{x}) \qquad(4) $$

Notice that the function we are fitting now is non-linear in $\mathbf{x}$ but we can still apply linear regression in the same way as before because the function is still linear in both $\phi(\mathbf{x})$ and $\mathbf{w}$.

Use scikit-learn [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn.preprocessing.PolynomialFeatures) to fit a second order polynomial to the data, plot the fit line.

In [0]:
# write your code here


Increase the order of the polynomial. When does the model start to overfit? Which order of polynomial would you use?

## 2) Neural Network for Image Classification
In this section we are going to recap the ideas behind deep neural networks and implement a network to classify cat images (I can sense the excitement). 

The dataset we will use is a set of labeled images containing cats (label=1) and non-cats (label=0). Run the cell below to load in train and test splits of the dataset from the local `data.h5` file. Make sure you have downloaded this data file in addition to the notebook from the lab repository.

In [0]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

The following cell displays an image in the dataset - change the index and re-run the cell to see other images.

In [0]:
# Example of a cat picture
index = 208
plt.imshow(train_x_orig[index])
print (f'y = {train_y[0,index]}. It\'s a {classes[train_y[0,index]].decode("utf-8")}!')

### 2.1) Explore the dataset
Print the values of: 
- a) number of training examples (`num_train`)
- b) number of test examples (`num_test`)
- c) height/width of image or the number of pixels (`num_px`). 

Note, `train_set_x_orig` is a numpy-array of shape (`num_train`, `num_px`, `num_px`, 3).

In [0]:
# write your code here


### 2.2) Reshape the images
Reshape the training (`train_x_orig`) and test (`test_x_orig`) data sets so that each image is flattened into column vector.

<img src="images/imvectorkiank.png" style="width:450px;height:300px;">

<caption><center> <u>Figure 1</u>: Image to vector conversion. <br> </center></caption>

Print the shape of the reshaped training and testing datasets.

In [0]:
# write your code here


### 2.3) Standardise the images
The pixel value is a vector of three numbers (representing the RGB channels) ranging from 0 to 255. A common preprocessing step in machine learning is to standardise your dataset (subtract the mean and then divide by the standard deviation). For picture datasets, it is simpler and more convenient to apply min-max normalisation by dividing every value by 255.

Apply min-max normalization to the dataset and check the minimum and maximum are 0 and 1, respectively.

In [0]:
# write your code here


### 2.4) Logistic regression classifier 
Logistic regression, despite its name, is a linear model for classification. The probabilities describing the possible outcomes of a classification are modeled using a logistic (sigmoid) function.

$$sigmoid(x) = \frac{1}{1+e^{-x}}$$

Logistic regression can be thought of as a neural network with a single node with a sigmoid activation function.

<img src="images/LogReg_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Figure 2</u>: Logistic regression classifier.</center></caption>

Use scikit-learn's [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic#sklearn.linear_model.LogisticRegression) to train a cat classifier. What's the classifier's accuracy on the training and test sets?

Hint, the classifier `fit` method has the following inputs:
- Training data with shape (`n_samples`, `n_features`)
- Target values with shape (`n_samples`,). Use `.flatten()` to collapse a 2-D array to a 1-D array.

In [0]:
# write your code here


### Neural network architecture
We will initially build a fully connected neural network with one hidden layer (i.e. one layer between input and output). When using more than one hidden layer we define it as a deep neural network.

<img src="images/2layerNN_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Figure 3</u>: 2-layer neural network.</center></caption>
    

- ***INPUT:*** $ x = [x_0,x_1,...,x_{12287}] \quad x_i \in [0,1] $

The input is a (64,64,3) image which we have already flattened to a vector of size (12288,1) and standardised.

- ***LINEAR:*** $ z^{[1]} = W^{[1]} x + b^{[1]} $

The input vector is multiplied by the weight matrix $W^{[1]}$ of size $(n^{[1]}, 12288)$ and then a bias term is added in a linear transformation. $n^{[1]}$ is the number of neurons in the hidden layer.

- ***RELU:*** $ a^{[1]} = RELU(z^{[1]}) = max(0,z^{[1]}) $

A non-linear activation function is then applied, in this case a rectified linear unit (or ReLU which outputs the maximum of the input and 0).

- ***LINEAR:*** $ z^{[2]} = W^{[2]} a^{[1]} + b^{[2]} $

A linear transformation is applied to the output of the hidden layer $ a^{[1]} $.

- ***SIGMOID:*** $ \hat{y} = a^{[2]} = \sigma(z^{[2]}) = \frac{1}{1 + e^{-z^{[2]}}} $

Given it's a binary classification task (cat or no cat) then the sigmoid or logistic function is the activation of the output layer (this is automatically selected by scikit-learn). 

- ***OUTPUT:*** $ \hat{y} $

The output is the probability the photo contains a cat so if the value is greater than 0.5 the prediction is cat.

The overall process from inputs to outputs is known as forward propagation, see Bishop section 5.1 for more information.

### Training a neural network
Neural networks are trained by learning the weights $W$ and biases $b$ in the hidden and output layers such that the network outputs the correct labels as optimally as possible. How well the network is performing is defined by a loss function, here the log loss (also called logistic regression loss or cross-entropy loss). The cost function $J$ which drives training is the sum of all the errors (log losses) for all the training examples in the training set.

$$ J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(\hat{y}^{(i)})+(1-y^{(i)})\log(1-\hat{y}^{(i)}) $$

The goal of neural network optimisation is to learn weights and biases that minimise the cost function. This is done by backpropagating the cost function error from the output layer, through the network to the first hidden layer. During this process the weights and biases are updated by gradient decent optimisation (using the cost function gradient with respect to all weights and bias).

For a parameter $\theta$, a simple gradient decent update rule is $ \theta = \theta - \eta \text{ } d\theta$, where $\eta$ is the learning rate (see Bishop section 5.2 and 5.3 for more information). More complex optimisation algorithms exist, for example the popular [Adam optimiser](https://arxiv.org/pdf/1412.6980).

### 2.4) Implement the neural network in scikit-learn
Thankfully, we don't need to construct the neural network manually and instead can use scikit-learn's [multi-layer perceptron (MLP) classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

Construct and train a neural network using MLP classifier with the following hyperparameters:
- Single hidden layer with 64 neurons.
- RELU activation function
- Stochastic gradient descent optimiser
- Initial learning rate $ \eta = 0.001 $ ($1e-3$)
- No regularisation $ \alpha = 0 $

Plot the loss curve (using `clf.loss_curve_`) which shows the network learning as the number of iteration increases. 

Has the loss curve flat-lined? Hint, you may have to specify the `max_iter` and `n_iter_no_change` parameters.

In [0]:
# write your code here


### 2.5 Evaluation
How does the neural network perform on the training and test data? Print the training and testing accuracy. Is classifier overfitting?

A confusion matrix is a way to visualise the performance of a classification model by showing the counts of the predicted and actual labels. The following terms are important metrics in classification tasks:
- total number of positives in the dataset i.e. cat images (P)
- total number of negatives in the dataset i.e. non cat images (N)
- number of correct positive predictions (TP)
- number of correct negative predictions (TN)
- number of incorrect positive predictions (FP)
- number of incorrect negative predictions (FN)
- accuracy $= \frac{TP+TN}{P+N}$
- sensitivity, recall or true positive rate $= \frac{TP}{P}$
- specificity or true negative rate $= \frac{TN}{N}$
- precision $= \frac{TP}{TP+FP}$




Plot the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html#sklearn.metrics.plot_confusion_matrix) for the test data. Where is the classifier making mistakes?

In [0]:
# write your code here


### Hyperparameter tuning using cross-validation
The performance of a neural network after training is highly dependent on how the hyperparameters are chosen. In contrast to the network parameters, $W$ and $b$, a hyperparameter refers to something that is fixed (usually manually chosen by the person training the model) throughout training and used to control the training process. Hyperparameters include the number of hidden layers, number of neurons in the hidden layers, learning rate, mini-batch size (for stochastic gradient descent optimisers) and regularisation parameter $\alpha$.

Cross-validation (CV for short) is used to evaluate model performance for model selection and to tune hyperparameters. In k-fold CV the training set is split into k smaller sets (called folds) and each fold is used as a validation set for models trained on all the other folds.

<img src="images/kfold_cv.png" style="width:450px;height:300px;">
<caption><center> <u>Figure 3</u>: 5 fold cross validation.</center></caption>

The performance of a model is measured by the average score (error) for all the fold. A test set should still be held out for final evaluation. For more information on CV see Bishop section 1.3.

Scikit-learn offers two approaches to search the hyperparameter space using cross validation: [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) which considers all parameter combinations and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) which samples a given number of candidates from a parameter space with a specified distribution.

### 2.6) Tune the learning rate and regularisation parameter using CV
The learning rate $\eta$ is an important hyperparameter to tune. Choosing a value that is too small will result in training that takes too long to converge and a value too large will cause instabilities in the training that prevent convergence. The regularisation parameter $\alpha$ controls the weighting of L2 regularisation in the cost function to help with overfitting by encouraging smaller weights leading to a smoother decision boundary.

Use RandomizedSearchCV to perform hyperparameter tuning of learning rate and regularisation parameter (limit the number of iterations `n_iter` ~10 and number of folds `cv` ~ 3 keep to reasonable training times).

A good methodology is to start with a wide range of hyperparameter values before homing in over a finer range.

Hint, use [numpy.logspace](https://numpy.org/doc/stable/reference/generated/numpy.logspace.html) to define the range of the hyperparameters spaced over a log scale for example:
```Python
alphas = np.logspace(-3, 0, 100)
learning_rates = np.logspace(-4, -2, 100)
```

In [0]:
# write your code here


It is common to create deep networks for most tasks with complexity controlled not by the network size (number of layers and nodes per layer) but with regularisation. Regularisation is a general term describing ways to control the complexity of a neural network in order to avoid overfitting. We have already discussed (and tuned) the L2 regularisation term of the error function.

Dropout is another regularisation technique and has been shown as an effective way of preventing overfitting but is unfortunately not implemented scikit-learn MLP. For dropout it's recommended to use either the [PyTorch](https://pytorch.org/tutorials/) or [TensorFlow/Keras](https://www.tensorflow.org/tutorials) frameworks which offer far more flexibility in neural network architectures as opposed to scikit-learn and faster training on GPUs.

Can you improve the performance of the network by tuning the various hyperparameters?

### 2.7) Just for fun, test your classifier with your own image
Upload a photo into the images folder and change the `my_image` variable in the cell below.

In [0]:
my_image = "cat.jpg" # change this to the name of your image file 

fname = "images/" + my_image
image = np.array(imageio.imread(fname))
my_image = np.array(Image.fromarray(image).resize((height,width))).reshape((-1,1))
my_image = my_image/255.
my_predicted_image = nn_clf.predict(my_image.T)

plt.imshow(image)
print ("Your neural network predicts a " + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  " picture.")

## 3) Support vector machines (SVM)
The algorithms implemented so far are parametric where a fixed number parameters are learn using training data which is then discarded and the learnt model used to make prediction on new data. In this section we implement support vector machines, a nonparametric model where the number of parameters are not fixed in advanced but grow with the amount of data.

A support vector machine constructs a hyper-plane to separate classes by maximising the distance (or margin) to the nearest training data points of any class. The figure below shows the decision boundary for linearly separable data, with three instances on the margin boundaries, called “support vectors”.

<img src="images/svm.png" style="width:450px;height:300px;">
<caption><center> <u>Figure 4</u>: SVM decision boundary, margin and support vectors.</center></caption>

For data that is not linearly separable, SVM classification involves quadratic programming optimisation to maximise the margin while incurring a penalty for each sample within the margin or a misclassification. The penalty strength is controlled by the hyperparameter `C`, a regularisation parameter that scales inversely.

SVMs can perform non-linear classification using the kernel trick (because the dual form of the quadratic optimisation problem consists of the pairwise dot product between training data points). The kernel function transforms the non-linear data into a feature space in which the linear support vector regression model is fit. 

For more information on SVMs, see Bishop section 7.1.

The exercise for SVM classification involves a toy dataset generated in the cell below:

In [0]:
np.random.seed(0)
N = 100
x1 = np.linspace(-4,3.5,N).reshape(-1,1) + np.random.normal(0,0.2,N).reshape(-1,1)
y1 = np.exp(0.6*x1) - 1 + np.random.normal(0,1.5,N).reshape(-1,1)
x2 = np.random.normal(-1, 1.5, N).reshape(-1,1)
y2 = np.random.normal(4, 1.5, N).reshape(-1,1)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(x1,y1)
ax.scatter(x2,y2)
ax.set_xlim([-5,4])
ax.set_ylim([-2,7])
ax.set_xlabel('X')
ax.set_ylabel('y')

### Logistic regression classifier
A logistic regression classifier has been implemented on the toy dataset below, clearly not suitable for the non-linear data.

In [0]:
X = np.concatenate((np.concatenate((x1,x2)), np.concatenate((y1,y2))), axis=1)
Y = np.concatenate((np.zeros(N),np.ones(N)))
# blue = 0, orange = 1

def plot_clf(clf):
    h = 0.01
    xx, yy = np.meshgrid(np.arange(-5, 4, h), np.arange(-2, 7, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired, shading='auto')
    ax.scatter(x1,y1)
    ax.scatter(x2,y2)
    ax.set_xlim([-5,4])
    ax.set_ylim([-2,7])
    ax.set_xlabel('X')
    ax.set_ylabel('y')
    
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, Y)
print(f'Logistic regression classifier accuracy: {logreg.score(X, Y)}')
plot_clf(logreg)

### 3.1) SVM classifier
Implement a SVM classifier using scikit-learn [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) and the `plot_clf` function defined in the previous cell.

Experiment with the hyperparameter `C` considering overfitting and generalisation. How would you tune `C`?

In [0]:
# write your code here


## 4) Classifier comparison
We are now going to compare the classifiers introduced above on a toy dataset plotted below. 

In [0]:
X, y = make_moons(noise=0.3, random_state=0)

fig, ax = plt.subplots(figsize=(6, 4))
# Plot the points
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright, edgecolors='k')
ax.set_title("Input data")

The task is to train 4 classifiers on the dataset and compare the respective decision boundaries using a [contour](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.contourf.html) plot, as shown below in figure 5.

<img src="images/contour.png" style="width:800px;height:200px;">
<caption><center> <u>Figure 5</u>: Example contour plot.</center></caption>

In [0]:
names = ["Logistic Regression", "Neural Net", "Linear SVM", "RBF SVM"]
              
classifiers = [
    LogisticRegression(C=1e5),
    MLPClassifier(alpha=1, max_iter=1000),    
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)]

# create meshgrid
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# colour map (contourf cmap parameter)
cm = plt.cm.RdBu

def Z_mesh(clf):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]        
    return Z.reshape(xx.shape)

The classifiers, with initial hyperparameters, are defined above. To help with making the contour plot, a [meshgrid](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) has been defined along with a `Z_mesh` function that returns the decision probability or decision output for the classifier for every point in the meshgrid.

In [0]:
# write your code here


## Wrap up

This lab has covered quite a bit, let's recap:
- We first built a linear regression model from scratch by least squares fitting. Then, we used scikit-learn to fit the same function with only a couple lines of code and used the same methodology to fit a non-linear function after a polynomial transformation. 
- We then built a fully-connected neural network to classify cats and achieved a test set accuracy of over 70% even before hyperparameter tuning. Note that in the field of computer vision it is common to use convolutional neural network (CNN) architectures instead of fully-connected networks since they show superior performance for image classification. A randomised search cross validation method was presented for hyperparameter tuning although automatic tuning using [Bayesian optimisation](https://arxiv.org/abs/1206.2944) can produce better results in less time.
- Finally, Support Vector Machine classification was used to fit a non-linear decision boundary and overfitting was explored.
- So which is the best model to use? Sadly, the no free lunch theorem states there is no universally best model and the optimal model is problem dependent according to the validity of the model assumptions. Therefore, we need to split the dataset into training and testing sets (and validation sets or use cross-validation) to evaluate different models and hyperparameters.

### References
- COMS30035 Machine Learning lecture notes 1 & 2.
- Bishop Pattern Recognition and Machine Learning: Chapter 3 for linear regression, chapter 5 for neural networks and chapter 7 for support vector machines. 

#### Materials used to create the lab
- University of Edinburgh's Machine Learning and Pattern Recognition (MLPR) course
- Andrew Ng's Neural Networks and Deep Learning course on Coursera
